In [30]:
import re
import pymongo
import datetime

In [21]:
mongoClient = pymongo.MongoClient("mongodb://localhost:27017/")
temporalSpatialDb = mongoClient["temporal_spatial_vispub"]
temporalSpatialDbPaper = temporalSpatialDb['paper']

In [33]:
currentYear = ''
currentVenue = 'VIS'

def newItem(title, author, year, venue):
    # print(title)
    # print(author)
    # print(year)
    # print(venue)
    now = datetime.datetime.now(datetime.timezone.utc)
    temporalSpatialDbPaper.find_one_and_update(
        {"title_annotated": title, 
        "author_annotated": author,
        "conference": venue,
        "year": year},
    {
        "$set": {
            'last_update_date': now,
        }
    },
    upsert=True)

def getAuthors(str):
    return list(map(lambda x: re.sub(r'^and ', "", x.strip()), authorStr.split(',')))

with open("data/vis2021-2017.txt") as f:
    title = ''
    authors = []
    titleCnt = 0
    for i in f.readlines():
        r = re.match(r'^IEEE VIS (\d+)', i)
        if (r):
            print(r.group(1))
            currentYear = int(r.group(1))
            continue
        if re.match(r'^\s+$', i):
            continue
        i = i.strip()
        
        if currentYear == 2021:
            title, authorStr = i.split('.')[0:2]
            title = title.strip()
            authors = getAuthors(authorStr)
            newItem(title, authors, currentYear, currentVenue)
        else:
            r = re.match(r'^Authors:\s*(.+)$', i)
            if r:
                authorStr = r.group(1)
                authors = getAuthors(authorStr)
                titleCnt -= 1
                if titleCnt != 0:
                    raise Exception("title-author unmatch", i)
                newItem(title, authors, currentYear, currentVenue)
                
            else:
                title = i.strip()
                titleCnt += 1
                if titleCnt > 1:
                    raise Exception("title-author unmatch!", i)
        


2021
2020
2019
2018
2017
